In [ ]:
import sys
from pathlib import Path
# set the notebook's CWD to your repo root
%cd D:/deepdemand
ROOT = Path.cwd().parents[0]   # go up one level
sys.path.insert(0, str(ROOT))


In [18]:
#!/usr/bin/env python3
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio.windows import from_bounds
from rasterio.enums import Resampling
from pathlib import Path
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D

# ========================= INPUTS =========================
BASEMAP_TIF   = "data/basemap/GBOverview.tif"  # assume already EPSG:27700
EDGES_GEOJSON = "data/highway_network/uk_driving_edges_simplified.geojson"

# England-ish bbox in EPSG:27700
ENGLAND_BBOX_27700 = (0, 0, 700000, 700000)  # (xmin, ymin, xmax, ymax)

PRED_DIR      = "projection"
BASELINE_YEAR = 2022
YEARS         = [2030, 2035, 2040, 2045]
PRED_JSON     = {y: f"{PRED_DIR}/prediction_{y}.json" for y in [BASELINE_YEAR] + YEARS}

OUT_DIR_PCT = Path(PRED_DIR) / f"maps_pct_change_vs_{BASELINE_YEAR}"
OUT_DIR_RAW = Path(PRED_DIR) / f"maps_raw_change_vs_{BASELINE_YEAR}"
OUT_DIR_PCT.mkdir(parents=True, exist_ok=True)
OUT_DIR_RAW.mkdir(parents=True, exist_ok=True)

# Legend output (standalone)
LEGEND_DIR = Path(PRED_DIR) / "legends"
LEGEND_DIR.mkdir(parents=True, exist_ok=True)
OUT_LEG_PCT = LEGEND_DIR / "legend_pct_change.svg"
OUT_LEG_RAW = LEGEND_DIR / "legend_raw_change.svg"

# ========================= Styling =========================
LINEWIDTH = 4
FIGSIZE   = (10, 12)
BASEMAP_MAX_PIX = 2000
EPS_DENOM = 1e-6
ALPHA_MIN, ALPHA_MAX = 0.2, 1.0

BASEMAP_ALPHA = 0.5

# Colors (use your two)
COLOR_TRAIN = (31/255, 119/255, 180/255)  # blue
COLOR_TEST  = (214/255, 39/255, 40/255)   # red

# If edges geojson CRS is missing, assume lon/lat
EDGES_ASSUME_CRS_IF_MISSING = "EPSG:4326"

# ========================= HELPERS =========================
def read_basemap_crop_27700(tif_path: str, bbox_27700, max_pix: int):
    xmin, ymin, xmax, ymax = bbox_27700
    with rasterio.open(tif_path) as src:
        if src.crs is None or src.crs.to_epsg() != 27700:
            raise ValueError("Basemap must have CRS EPSG:27700 (or set/warp it beforehand).")

        win = from_bounds(xmin, ymin, xmax, ymax, transform=src.transform).round_offsets().round_lengths()
        win = win.intersection(rasterio.windows.Window(0, 0, src.width, src.height))

        scale = max(win.width / max_pix, win.height / max_pix, 1.0)
        out_w = int(max(1, win.width / scale))
        out_h = int(max(1, win.height / scale))

        data = src.read(
            window=win,
            out_shape=(src.count, out_h, out_w),
            resampling=Resampling.bilinear,
        )
        img = np.transpose(data, (1, 2, 0))

        left, bottom, right, top = rasterio.windows.bounds(win, src.transform)
        extent = (left, right, bottom, top)
        return img, extent


def ensure_27700(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    if gdf.crs is None:
        gdf = gdf.set_crs(EDGES_ASSUME_CRS_IF_MISSING)
    if gdf.crs.to_epsg() != 27700:
        gdf = gdf.to_crs(27700)
    return gdf


def load_pred_json_as_df(path: str) -> pd.DataFrame:
    with open(path, "r") as f:
        arr = json.load(f)

    u_list, v_list, k_list, preds = [], [], [], []
    for r in arr:
        eid = str(r["edge_id"])
        parts = eid.split("_")
        if len(parts) < 3:
            raise ValueError(f"Bad edge_id format (expected u_v_key): {eid}")
        u_list.append(int(parts[0]))
        v_list.append(int(parts[1]))
        k_list.append(int(parts[2]))
        preds.append(float(r["pred"]))

    df = pd.DataFrame({"u": u_list, "v": v_list, "key": k_list, "pred": preds})
    return df.groupby(["u", "v", "key"], as_index=False)["pred"].mean()


def alpha_from_value(val: np.ndarray, cap: float) -> np.ndarray:
    cap = float(max(cap, 1e-12))
    a = np.clip(np.abs(val) / cap, 0.0, 1.0)
    return ALPHA_MIN + (ALPHA_MAX - ALPHA_MIN) * a


def _geom_to_segments(geom):
    if geom is None or geom.is_empty:
        return []
    gt = geom.geom_type
    if gt == "LineString":
        return [np.asarray(geom.coords, dtype=float)]
    if gt == "MultiLineString":
        return [np.asarray(g.coords, dtype=float) for g in geom.geoms if (g is not None and not g.is_empty)]
    return []


def add_lines_with_per_feature_alpha(ax, gdf, color_rgb, alphas, linewidth):
    segs, cols = [], []
    for geom, a in zip(gdf.geometry.values, alphas):
        for s in _geom_to_segments(geom):
            if s.shape[0] >= 2:
                segs.append(s)
                cols.append((color_rgb[0], color_rgb[1], color_rgb[2], float(a)))
    if not segs:
        return
    lc = LineCollection(segs, colors=cols, linewidths=linewidth, capstyle="round", joinstyle="round")
    ax.add_collection(lc)


def save_standalone_legend_svg(out_path: Path, title: str, pos_label: str, neg_label: str):
    """
    Standalone legend SVG showing:
    - red (positive), blue (negative)
    - alpha meaning
    """
    fig, ax = plt.subplots(figsize=(4.2, 2.0))
    ax.axis("off")

    handles = [
        Line2D([0], [0], color=COLOR_TEST,  lw=4, label=pos_label),
        Line2D([0], [0], color=COLOR_TRAIN, lw=4, label=neg_label),
        Line2D([0], [0], color="black", lw=4, alpha=ALPHA_MIN, label=f"Min magnitude (alpha={ALPHA_MIN:.1f})"),
        Line2D([0], [0], color="black", lw=4, alpha=ALPHA_MAX, label=f"Max magnitude (alpha={ALPHA_MAX:.1f})"),
    ]
    leg = ax.legend(handles=handles, title=title, frameon=False, loc="center left", fontsize=10, title_fontsize=11)
    fig.tight_layout()
    fig.savefig(out_path, format="svg")
    plt.close(fig)
    print("[Save legend]", out_path)


# ========================= LOAD DATA =========================
print("[Load] Basemap:", BASEMAP_TIF)
base_img, base_extent = read_basemap_crop_27700(BASEMAP_TIF, ENGLAND_BBOX_27700, BASEMAP_MAX_PIX)

print("[Load] Edges:", EDGES_GEOJSON)
# edges = gpd.read_file(EDGES_GEOJSON)
edges = ensure_27700(edges)

required_cols = {"u", "v", "key"}
missing = required_cols - set(edges.columns)
if missing:
    raise ValueError(f"Edges geojson missing columns: {sorted(missing)}")

edges["u"] = edges["u"].astype(np.int64)
edges["v"] = edges["v"].astype(np.int64)
edges["key"] = edges["key"].astype(np.int64)

# Crop edges to bbox (speed + less blank space)
xmin, ymin, xmax, ymax = ENGLAND_BBOX_27700
edges = edges.cx[xmin:xmax, ymin:ymax].copy()

print("[Load] Baseline predictions:", PRED_JSON[BASELINE_YEAR])
base_df = load_pred_json_as_df(PRED_JSON[BASELINE_YEAR]).rename(columns={"pred": "pred_base"})

# ========================= PER-YEAR PROCESSING =========================
for y in YEARS:
    print(f"\n=== Year {y} vs {BASELINE_YEAR} ===")
    fut_df = load_pred_json_as_df(PRED_JSON[y]).rename(columns={"pred": "pred_fut"})

    df = base_df.merge(fut_df, on=["u", "v", "key"], how="inner")
    if df.empty:
        print(f"[WARN] No overlapping predicted edges for {y}. Skipping.")
        continue

    g = edges.merge(df, on=["u", "v", "key"], how="inner")
    if g.empty:
        print(f"[WARN] No geometries matched (after bbox crop) for {y}. Skipping.")
        continue

    base = np.clip(g["pred_base"].to_numpy(dtype=float), EPS_DENOM, None)
    fut  = g["pred_fut"].to_numpy(dtype=float)

    d_raw = fut - base
    d_pct = 100.0 * (fut - base) / base

    pct_min, pct_max = float(np.nanmin(d_pct)), float(np.nanmax(d_pct))
    raw_min, raw_max = float(np.nanmin(d_raw)), float(np.nanmax(d_raw))
    print(f"[Stats] % change min={pct_min:.3f} max={pct_max:.3f}")
    print(f"[Stats] raw change min={raw_min:.3f} max={raw_max:.3f}")

    cap_pct = float(max(abs(pct_min), abs(pct_max), 1e-12))
    cap_raw = float(max(abs(raw_min), abs(raw_max), 1e-12))

    g_pct = g.copy()
    g_pct["val"] = d_pct
    g_pct["alpha"] = alpha_from_value(d_pct, cap_pct)

    g_raw = g.copy()
    g_raw["val"] = d_raw
    g_raw["alpha"] = alpha_from_value(d_raw, cap_raw)

    # ---------- PCT PDF ----------
    out_pdf_pct = OUT_DIR_PCT / f"pct_change_{y}_vs_{BASELINE_YEAR}.pdf"
    fig, ax = plt.subplots(figsize=FIGSIZE)
    ax.imshow(base_img, extent=base_extent, origin="upper", alpha=BASEMAP_ALPHA)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

    pos = g_pct["val"].to_numpy() > 0
    neg = g_pct["val"].to_numpy() < 0

    if neg.any():
        add_lines_with_per_feature_alpha(
            ax, g_pct.loc[neg], color_rgb=COLOR_TRAIN,
            alphas=g_pct.loc[neg, "alpha"].to_numpy(), linewidth=LINEWIDTH
        )
    if pos.any():
        add_lines_with_per_feature_alpha(
            ax, g_pct.loc[pos], color_rgb=COLOR_TEST,
            alphas=g_pct.loc[pos, "alpha"].to_numpy(), linewidth=LINEWIDTH
        )

    ax.set_title(f"{y} vs {BASELINE_YEAR} – Percent change (alpha scaled to ±{cap_pct:.2f}%)")
    ax.set_axis_off()
    fig.tight_layout()
    fig.savefig(out_pdf_pct, bbox_inches="tight")
    plt.close(fig)
    print("[Save]", out_pdf_pct)

    # ---------- RAW PDF ----------
    out_pdf_raw = OUT_DIR_RAW / f"raw_change_{y}_vs_{BASELINE_YEAR}.pdf"
    fig, ax = plt.subplots(figsize=FIGSIZE)
    ax.imshow(base_img, extent=base_extent, origin="upper", alpha=BASEMAP_ALPHA)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

    pos = g_raw["val"].to_numpy() > 0
    neg = g_raw["val"].to_numpy() < 0

    if neg.any():
        add_lines_with_per_feature_alpha(
            ax, g_raw.loc[neg], color_rgb=COLOR_TRAIN,
            alphas=g_raw.loc[neg, "alpha"].to_numpy(), linewidth=LINEWIDTH
        )
    if pos.any():
        add_lines_with_per_feature_alpha(
            ax, g_raw.loc[pos], color_rgb=COLOR_TEST,
            alphas=g_raw.loc[pos, "alpha"].to_numpy(), linewidth=LINEWIDTH
        )

    ax.set_title(f"{y} vs {BASELINE_YEAR} – Raw change (alpha scaled to ±{cap_raw:.2f})")
    ax.set_axis_off()
    fig.tight_layout()
    fig.savefig(out_pdf_raw, bbox_inches="tight")
    plt.close(fig)
    print("[Save]", out_pdf_raw)


# ========================= Legends (standalone SVG) =========================
save_standalone_legend_svg(
    OUT_LEG_PCT,
    title="Percent change legend",
    pos_label="Increase (positive change)",
    neg_label="Decrease (negative change)",
)

save_standalone_legend_svg(
    OUT_LEG_RAW,
    title="Raw change legend",
    pos_label="Increase (positive change)",
    neg_label="Decrease (negative change)",
)

print("\nDone.")

[Load] Basemap: data/basemap/GBOverview.tif
[Load] Edges: data/highway_network/uk_driving_edges_simplified.geojson
[Load] Baseline predictions: projection/prediction_2022.json

=== Year 2030 vs 2022 ===
[Stats] % change min=-100.000 max=96.990
[Stats] raw change min=-9750.838 max=9288.756
[Save] projection\maps_pct_change_vs_2022\pct_change_2030_vs_2022.pdf
[Save] projection\maps_raw_change_vs_2022\raw_change_2030_vs_2022.pdf

=== Year 2035 vs 2022 ===
[Stats] % change min=-100.000 max=111.870
[Stats] raw change min=-12304.555 max=7796.924
[Save] projection\maps_pct_change_vs_2022\pct_change_2035_vs_2022.pdf
[Save] projection\maps_raw_change_vs_2022\raw_change_2035_vs_2022.pdf

=== Year 2040 vs 2022 ===
[Stats] % change min=-100.000 max=194.978
[Stats] raw change min=-8640.450 max=21411.695
[Save] projection\maps_pct_change_vs_2022\pct_change_2040_vs_2022.pdf
[Save] projection\maps_raw_change_vs_2022\raw_change_2040_vs_2022.pdf

=== Year 2045 vs 2022 ===
[Stats] % change min=-100.000 m

In [16]:
# --- Post-run diagnostics for YEAR=2040 ---
# Assumes the previous script has been run in the same Python session,
# so base_df, edges, EPS_DENOM, BASELINE_YEAR, PRED_JSON, OUT_DIR_PCT, etc. exist.

import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

YEAR = 2040
EDGE_TYPE_JSON = "data/traffic_volume/edge_to_highway_type.json"

# -------- build matched GeoDataFrame (same logic as in the map script) --------
fut_df_2040 = load_pred_json_as_df(PRED_JSON[YEAR]).rename(columns={"pred": "pred_fut"})
df_2040 = base_df.merge(fut_df_2040, on=["u", "v", "key"], how="inner")
if df_2040.empty:
    raise RuntimeError("No overlapping predicted edges for 2040 vs baseline.")

g_2040 = edges.merge(df_2040, on=["u", "v", "key"], how="inner")
if g_2040.empty:
    raise RuntimeError("No geometries matched for 2040 (after bbox crop).")

base = np.clip(g_2040["pred_base"].to_numpy(dtype=float), EPS_DENOM, None)
fut  = g_2040["pred_fut"].to_numpy(dtype=float)
pct  = 100.0 * (fut - base) / base

# -------- output paths --------
diag_dir = Path(PRED_DIR) / f"diagnostics_{YEAR}_vs_{BASELINE_YEAR}"
diag_dir.mkdir(parents=True, exist_ok=True)

out_hist = diag_dir / f"hist_pct_change_{YEAR}_vs_{BASELINE_YEAR}.pdf"
out_scatter = diag_dir / f"scatter_baseline_vs_{YEAR}.pdf"

# =========================
# 1) Histogram of % change
# =========================
plt.figure(figsize=(6.5, 4.2))
plt.hist(pct[np.isfinite(pct)], bins=60)
plt.xlabel(f"Percent change in predicted traffic volume ({YEAR} vs {BASELINE_YEAR})")
plt.ylabel("Number of edges")
plt.title(f"Distribution of percent change ({YEAR} vs {BASELINE_YEAR})")
plt.tight_layout()
plt.savefig(out_hist)
plt.close()
print("Saved:", out_hist)

# ==========================================
# 2) Scatter: baseline vs 2040 prediction
#     (UPDATED: point color by highway type)
# ==========================================

# -------- load edge -> highway type --------
with open(EDGE_TYPE_JSON, "r", encoding="utf-8") as f:
    edge_to_type = json.load(f)

# build edge_id to match your convention
edge_ids = (
    g_2040["u"].astype(str) + "_" +
    g_2040["v"].astype(str) + "_" +
    g_2040["key"].astype(str)
).to_numpy()

highway_types = np.array([edge_to_type.get(eid, "other") for eid in edge_ids], dtype=object)

# colors (same as your stacked histogram)
# Colors (match your convention)
COLORS = {
    "motorway":        "#1f77b4",  # blue
    "motorway_link":   "#98df8a",  # light green
    "trunk":           "#ff7f0e",  # orange
    "trunk_link":      "#ffbb78",  # light orange
}

types_unique = ["motorway", "motorway_link", "trunk", "trunk_link"]

plt.figure(figsize=(5.0, 4.5))

for t in types_unique:
    mask = (highway_types == t)
    if not np.any(mask):
        continue
    plt.scatter(
        base[mask],
        fut[mask],
        s=3,
        alpha=0.5,
        edgecolor="none",
        color=COLORS[t],
        label=t.replace("_", " ").title()
    )


# reference line y=x
mn = float(np.nanmin([base.min(), fut.min()]))
mx = float(np.nanmax([base.max(), fut.max()]))
plt.plot([mn, mx], [mn, mx], linestyle="--", linewidth=1)

plt.xlabel(f"Baseline predicted traffic volume ({BASELINE_YEAR})")
plt.ylabel(f"Predicted traffic volume ({YEAR})")
plt.legend(frameon=False, markerscale=2.5)
plt.tight_layout()
plt.savefig(out_scatter)
plt.close()
print("Saved:", out_scatter)

Saved: projection\diagnostics_2040_vs_2022\hist_pct_change_2040_vs_2022.pdf
Saved: projection\diagnostics_2040_vs_2022\scatter_baseline_vs_2040.pdf


In [4]:
# --- Stacked histogram of % change by highway type (YEAR = 2040) ---
# Assumes the previous diagnostics code has already run in the SAME session,
# so the following already exist:
#   - g_2040 (GeoDataFrame with pred_base, pred_fut)
#   - pct (numpy array of % change)
#   - PRED_DIR, BASELINE_YEAR, YEAR
#
# This script only adds highway-type stratification and plotting.

import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# -------- inputs --------
YEAR = 2040
EDGE_TYPE_JSON = "data/traffic_volume/edge_to_highway_type.json"

# -------- load edge -> highway type --------
with open(EDGE_TYPE_JSON, "r") as f:
    edge_to_type = json.load(f)

# build edge_id to match your convention
edge_ids = (
    g_2040["u"].astype(str) + "_" +
    g_2040["v"].astype(str) + "_" +
    g_2040["key"].astype(str)
).to_numpy()

highway_types = np.array(
    [edge_to_type.get(eid, "other") for eid in edge_ids],
    dtype=object
)

# percent change (already computed earlier)
pct_vals = pct

# -------- group by highway type --------
types_unique = [
    "motorway",
    "motorway_link",
    "trunk",
    "trunk_link",
]
data_by_type = [
    pct_vals[highway_types == t] for t in types_unique
]

# -------- colors (consistent with earlier convention) --------
COLORS = {
    "motorway":        (31/255, 119/255, 180/255),  # blue
    "motorway_link":   (174/255, 199/255, 232/255), # light blue
    "trunk":           (214/255, 39/255, 40/255),   # red
    "trunk_link":      (255/255, 152/255, 150/255), # light red
}

colors = [COLORS[t] for t in types_unique]

# -------- plot --------
out_dir = Path(PRED_DIR) / f"diagnostics_{YEAR}_vs_{BASELINE_YEAR}"
out_dir.mkdir(parents=True, exist_ok=True)
out_fig = out_dir / f"hist_pct_change_stacked_by_highway_{YEAR}.pdf"

plt.figure(figsize=(7.0, 4.5))

bins = np.linspace(
    np.nanmin(pct_vals),
    np.nanmax(pct_vals),
    60
)

plt.hist(
    data_by_type,
    bins=bins,
    stacked=True,
    color=colors,
    label=[t.replace("_", " ").title() for t in types_unique]
)

plt.xlabel(f"Percent change in predicted traffic volume ({YEAR} vs {BASELINE_YEAR})")
plt.ylabel("Number of edges")
plt.title(f"Stacked distribution of percent change by highway type ({YEAR})")
plt.legend(frameon=False)
plt.tight_layout()
plt.savefig(out_fig)
plt.close()

print("Saved:", out_fig)

Saved: projection\diagnostics_2040_vs_2022\hist_pct_change_stacked_by_highway_2040.pdf


In [8]:
#!/usr/bin/env python3
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# =========================
# Inputs
# =========================
BASELINE_YEAR = 2022
TARGET_YEAR   = 2040

PRED_BASE = f"projection/prediction_{BASELINE_YEAR}.json"
PRED_FUT  = f"projection/prediction_{TARGET_YEAR}.json"

EDGE_TYPE_JSON = "data/traffic_volume/edge_to_highway_type.json"

OUT_FIG = f"projection/diagnostics_2040_vs_2022/cdf_pct_change_{TARGET_YEAR}_by_highway_type.pdf"

# Colors (match your convention)
COLORS = {
    "motorway":        "#1f77b4",  # blue
    "motorway_link":   "#98df8a",  # light green
    "trunk":           "#ff7f0e",  # orange
    "trunk_link":      "#ffbb78",  # light orange
}

ORDER = ["motorway", "trunk", "motorway_link", "trunk_link"]

# =========================
# Helpers
# =========================
def load_pred(path):
    with open(path, "r") as f:
        rows = json.load(f)
    return pd.DataFrame({
        "edge_id": [str(r["edge_id"]) for r in rows],
        "pred":    [float(r["pred"])  for r in rows],
    })

def empirical_cdf(x):
    x = np.sort(x)
    y = np.arange(1, len(x) + 1) / len(x)
    return x, y

# =========================
# Load predictions
# =========================
df_base = load_pred(PRED_BASE).rename(columns={"pred": "pred_base"})
df_fut  = load_pred(PRED_FUT ).rename(columns={"pred": "pred_fut"})

df = df_base.merge(df_fut, on="edge_id", how="inner")

# % change (guard against zero)
eps = 1e-6
df["pct_change"] = 100.0 * (df["pred_fut"] - df["pred_base"]) / np.clip(df["pred_base"], eps, None)

# =========================
# Highway types
# =========================
with open(EDGE_TYPE_JSON, "r") as f:
    edge_type = json.load(f)

df["highway_type"] = df["edge_id"].map(edge_type)
df = df.dropna(subset=["highway_type"])

# =========================
# Plot CDF
# =========================
plt.figure(figsize=(5, 4.5))

for ht in ORDER:
    sub = df.loc[df["highway_type"] == ht, "pct_change"].to_numpy()
    if len(sub) == 0:
        continue

    x, y = empirical_cdf(sub)
    plt.plot(
        x, y,
        lw=2,
        color=COLORS.get(ht, "grey"),
        label=ht.replace("_", " ").title()
    )

plt.xlabel("Percentage change in predicted traffic volume")
plt.ylabel("Cumulative probability")
plt.xlim(np.nanpercentile(df["pct_change"], 1),
         np.nanpercentile(df["pct_change"], 99))
plt.ylim(0, 1)

plt.legend(frameon=False)
# plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(OUT_FIG)
plt.close()

print(f"[Saved] {OUT_FIG}")

[Saved] projection/diagnostics_2040_vs_2022/cdf_pct_change_2040_by_highway_type.pdf
